# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

In [32]:
df.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)

#new df
# 1. Create a new DataFrame that only includes customers who:
#    - have a **low total_claim_amount** (e.g., below $1,000),
#    - have a response "Yes" to the last marketing campaign.


df['Total Claim Amount'] = pd.to_numeric(df['Total Claim Amount'], errors='coerce')

def filter_values(row):
    if row['Response'] == 'Yes' and row['Total Claim Amount'] < 1000:
        return True
    return False
filtered_df = df[['Customer','Response','Total Claim Amount']]
filtered_df = df[df.apply(filter_values, axis=1)]

print(filtered_df)

# 2. Using the original Dataframe, analyze:
#    - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
#    - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

def filter_values_yes_only(row):
   return row['Response'] == 'Yes'

filtered_df2 = df[df.apply(filter_values_yes_only, axis=1)]
filtered_df2 = filtered_df2[['Customer','Response','Monthly Premium Auto','Customer Lifetime Value','Policy Type','Gender','Total Claim Amount']]

grouped_stats = filtered_df2.groupby(['Policy Type', 'Gender']).agg({
    'Monthly Premium Auto': 'mean',
    'Customer Lifetime Value': 'mean',
    'Total Claim Amount': 'mean'
}).reset_index()

grouped_stats['Avg Monthly Premium'] = grouped_stats['Monthly Premium Auto']
grouped_stats['Avg CLV'] = grouped_stats['Customer Lifetime Value']
grouped_stats['Avg Total Claim Amount'] = grouped_stats['Total Claim Amount']

grouped_stats.head()

# 3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.
filtered_df3 = df[['Customer', 'State']]
state_counts = filtered_df3['State'].value_counts().reset_index()
state_counts.columns = ['State','Customer Count']
state_counts = state_counts[state_counts['Customer Count'] > 500]

state_counts.head()

# 4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.
filtered_df4 = df[['Customer Lifetime Value', 'Education','Gender']]
grouped_stats2 = filtered_df4.groupby(['Education', 'Gender']).agg({
    'Customer Lifetime Value': ['max','min','median']

}).reset_index()
grouped_stats2.columns = ['Education', 'Gender','Max CLV','Min CLV','Median CLV']
grouped_stats2.head()

       Unnamed: 0 Customer       State  Customer Lifetime Value Response  \
3               3  XL78013      Oregon             22332.439460      Yes   
8               8  FM55990  California              5989.773931      Yes   
15             15  CW49887  California              4626.801093      Yes   
19             19  NJ54277  California              3746.751625      Yes   
27             27  MQ68407      Oregon              4376.363592      Yes   
...           ...      ...         ...                      ...      ...   
10844       10844  FM31768     Arizona              5979.724161      Yes   
10852       10852  KZ80424  Washington              8382.478392      Yes   
10872       10872  XT67997  California              5979.724161      Yes   
10887       10887  BY78730      Oregon              8879.790017      Yes   
10897       10897  MM70762     Arizona              9075.768214      Yes   

       Coverage             Education Effective To Date EmploymentStatus  \
3      Exte

,Education,Gender,Max CLV,Min CLV,Median CLV
0,Bachelor,F,73225.95652,1904.000852,5640.505303
1,Bachelor,M,67907.27050,1898.007675,5548.031892
2,College,F,61850.18803,1898.683686,5623.611187
3,College,M,61134.68307,1918.119700,6005.847375
4,Doctor,F,44856.11397,2395.570000,5332.462694
